# Package Loading

In [16]:
# Standard Library
import json
import os
import re

# Data Manipulation
import pandas as pd

# PDFs
import pdfplumber

# LLMs
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.graphs.graph_document import (
    GraphDocument,
    Node as BaseNode,
    Relationship as BaseRelationship,
)
from langchain_huggingface import HuggingFaceEmbeddings
from openai import OpenAI
import tiktoken 

# Neo4j
from neo4j import GraphDatabase
from neo4j.exceptions import AuthError

# Typing & Validation
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional

# Raw Data Extraction

In [ ]:
# Define the directory containing PDFs
pdf_directory = "./reports_ie/"

# List all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith(".pdf")]

for file in pdf_files:
    print(file)

IE-10375 - 210827 Collision with track equipment.pdf
IE-10397 - 211207 Clontarf.pdf
IE-10404 - 230222 Broken Rail Emly.pdf
IE-200608 BnM Collision LC Offaly.pdf
IE-6218-200111 Collision RRME Rosslare.pdf
IE-6262-200429 LC Collision XM240.pdf
IE-6291-200524 LC XA068 Ashfield.pdf
IE-6305 - 200707_locomotive_224.pdf


In [153]:
# Specify file name that you want to process
pdf_name = "IE-10404 - 230222 Broken Rail Emly.pdf"

In [156]:
# Define regex patterns to identify the start of the summary and contents sections
SUMMARY_PATTERN = re.compile(r"^summary", re.IGNORECASE)
CONTENTS_PATTERN = re.compile(r"^contents", re.IGNORECASE)

# Define function to extract the summary section from Ireland reports
def extract_summary_section(pdf_path: str, header_lines: int = 1) -> str:
    """
    Extracts text from 'Summary' to 'Contents' in an Irish rail report PDF.
    If no summary section is found, returns the text from all pages.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"File not found: {pdf_path}")

    summary_text = ""
    full_text = ""
    capturing = False  

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text_lines = page_text.split("\n")
                text_without_header = text_lines[header_lines:]
                page_content = f"[Page {page.page_number}]\n" + "\n".join(text_without_header) + "\n\n"

                # Always append to full_text
                full_text += page_content

                if text_without_header:
                    first_line = text_without_header[0].strip().lower()

                    if SUMMARY_PATTERN.match(first_line):
                        capturing = True
                    
                    if CONTENTS_PATTERN.match(first_line) and capturing:
                        # Stop capturing when "Contents" is found after summary started.
                        break
                    
                    if capturing:
                        summary_text += page_content

    if not summary_text:
        print(f"Warning: No summary section found in {pdf_path}. Returning full text.")
        return full_text

    return summary_text

# Example usage:
pdf_text = extract_summary_section(f"./reports_ie/{pdf_name}", header_lines=1)
print(pdf_text)

[Page 6]
Summary
1 On Tuesday 21st February 2023 a T3 Possession was organised on the Up and Down
lines of the Dublin to Cork mainline to allow for a track section, near Emly Level Crossing
(LC), to undergo track maintenance.
2 As part of the track maintenance, stressing and welding of the rails had to be undertaken
in preparation for ballast cleaning.
3 The stressing of the rails was being carried out on the Up line, which involved cutting both
rails which was marked by a Iarnród Éireann Infrastructure Manager (IÉ-IM) staff member,
the Person in Charge of Stressing (this member of IÉ-IM staff was also the Track Delivery
Unit Engineer (TDU Engineer) who supervised the works).
4 A welding contractor was engaged to carry out the welding at the site location (110 miles
355 yards), with a team comprising of a lead and second welder (the Welders) and a Weld
Supervisor. Prior to the welding, the Welders placed clamps on either side of the first cut
rail section and attached Rail Tensors to p

# Langchain Chunk Splitting

In [157]:
def split_text_into_chunks(text: str, chunk_size: int = 2000, chunk_overlap: int = 300) -> list[str]:
    """
    Splits text into smaller overlapping chunks using LangChain's text splitter.
    """
    if not text:
        print("Warning: No text provided for splitting.")
        return []

    if len(text) <= chunk_size:
        return [text]  # If text is smaller than chunk size, return as single chunk

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

# Split the extracted text
text_chunks = split_text_into_chunks(pdf_text)

# Print the number of chunks and a sample chunk
print(f"Total chunks: {len(text_chunks)}\nFirst chunk:\n{text_chunks[0]}")

Total chunks: 63
First chunk:
[Page 6]
Summary
1 On Tuesday 21st February 2023 a T3 Possession was organised on the Up and Down
lines of the Dublin to Cork mainline to allow for a track section, near Emly Level Crossing
(LC), to undergo track maintenance.
2 As part of the track maintenance, stressing and welding of the rails had to be undertaken
in preparation for ballast cleaning.
3 The stressing of the rails was being carried out on the Up line, which involved cutting both
rails which was marked by a Iarnród Éireann Infrastructure Manager (IÉ-IM) staff member,
the Person in Charge of Stressing (this member of IÉ-IM staff was also the Track Delivery
Unit Engineer (TDU Engineer) who supervised the works).
4 A welding contractor was engaged to carry out the welding at the site location (110 miles
355 yards), with a team comprising of a lead and second welder (the Welders) and a Weld
Supervisor. Prior to the welding, the Welders placed clamps on either side of the first cut
rail section 

# Relevant Chunk Retrieval

### Vector-based Retrieval

In [158]:
# Define embeddings 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [159]:
# Store text chunks into FAISS vector store
vectorstore = FAISS.from_texts(text_chunks, embeddings)

print(f"Stored {len(text_chunks)} chunks in FAISS.")

Stored 63 chunks in FAISS.


In [165]:
# Define entities of interest that you'd like to extract chunks for from the vector store
entities_of_interest = ["unique accident", "accident type", "date", "time", "country", "regulatory body"]

# Function for extracting most relevant chunks from vector store
def find_most_relevant_chunks(entities: list[str], top_k: int) -> str:
    """
    Finds the most relevant text chunks for each entity of interest
    using FAISS similarity search and removes duplicates (if same chunk retrieved).
    
    Args:
    - entities (list): List of entity names to query (e.g., ["date", "location"])
    - top_k (int): Number of chunks to retrieve per entity
    
    Returns:
    - unique_relevant_chunks (list): Deduplicated relevant chunks
    """
    retrieved_chunks = set()  # Use a set to avoid duplicate chunks

    for entity in entities:
        print(f"Searching for entity: {entity}")
        query = f"Extract details specifically about {entity}. Focus on structured information."
        found_chunks = vectorstore.similarity_search(query, k=top_k)

        for chunk in found_chunks:
            retrieved_chunks.add(chunk.page_content)  # Add chunk if not already present

    # Convert set back to a list and join into a single string
    unique_relevant_chunks = list(retrieved_chunks)
    combined_text = "\n".join(unique_relevant_chunks)

    print(f"\nFound {len(unique_relevant_chunks)} unique relevant chunks.")
    return combined_text

# Find & combine relevant chunks
relevant_text = find_most_relevant_chunks(entities_of_interest, top_k=3)

print(f"\nMost Relevant Chunks Combined:\n{relevant_text}")

Searching for entity: unique accident
Searching for entity: accident type
Searching for entity: date
Searching for entity: time
Searching for entity: country
Searching for entity: regulatory body

Found 7 unique relevant chunks.

Most Relevant Chunks Combined:
[Page 44]
165 The CWR Record Sheet in CCE-TMS-323 has no sections for the Person in Charge
of Stressing to record the serial number and date of calibration/ recalibration of the Rail
Tensor being used, the pressure reading on the pressure gauges and the frequency of the
checks on the pressure gauges. As a result, the RAIU make the following safety
recommendation to address AO-02 and AO-03 (paragraph 149):
Safety Recommendation 2024002-04
IÉ-IM CCE should review and update the CWR Record Sheet to include sections for
recording the serial number and calibration/ recalibration date of the Rail Tensors
being used, the pressure gauge readings and the frequency of these checks on the
pressure gauges. On completion, ensure staff are bri

## Instantiating GPT

In [38]:
# Set the API key and model name
MODEL="gpt-4o-mini"

# Load OpenAI API Key from requirements file
with open("gpt-personal-key.txt", "r") as file:
    OPENAI_API_KEY = file.read().strip()

# Instantiate OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

## Token Count

In [43]:
# Function for calculating tokens
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    """Efficiently counts tokens in a text for a given OpenAI model."""
    if model not in count_tokens.encoders:
        count_tokens.encoders[model] = tiktoken.encoding_for_model(model)
    return len(count_tokens.encoders[model].encode(text))

count_tokens.encoders = {}

## Entity Extraction

In [44]:
# Define classes for the entities extraction
class Property(BaseModel):
    """A single property consisting of key and value."""
    key: str = Field(..., description="Property key")
    value: str = Field(..., description="Property value")

class Node(BaseNode):
    """Represents an entity in the railway accident knowledge graph."""
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    """Represents a relationship between two entities in the graph."""
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """A knowledge graph storing railway accident data."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [51]:
def call_gpt(prompt, temperature=1):
    """
    Calls the GPT model with the structured prompt and returns the raw response.
    """
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an expert in analyzing railway accident reports. Return output in JSON format only."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    
    response_text = completion.choices[0].message.content.strip()
    response_text = re.sub(r'^```json\n?|```$', '', response_text).strip()
    
    return response_text

In [142]:
def build_prompt(text):
    """
    Constructs a structured prompt to extract entities and relationships for railway accidents.
    """
    
    schema_example = """
    {
        "nodes": [
            {"id": "Dublin-Cork Accident", "type": "UniqueAccident"},
            {"id": "Train Derailment", "type": "AccidentType"},
            {"id": "23/12/2021", "type": "Date"},
            {"id": "16:32", "type": "Time"},
            {"id": "Ireland", "type": "Country"},
            {"id": "European Rail Agency", "type": "RegulatoryBody"}
        ],
        "rels": [
            {"source": "Dublin-Cork Accident", "target": "Ireland", "type": "occurred_in"},
            {"source": "Dublin-Cork Accident", "target": "", "type": "is_type"},
            {"source": "Dublin-Cork Accident", "target": "European Rail Agency", "type": "investigated_by"},
            {"source": "Dublin-Cork Accident", "target": "23/12/2021", "type": "has_date"},
            {"source": "Dublin-Cork Accident", "target": "16:32", "type": "has_time"}
        ]
    }
    """

    return f"""
    Analyze the following railway accident report context and extract structured knowledge.

    Return a JSON object with:
    - `nodes`: A list of entities, specifically {entities_of_interest}.
    - `rels`: A list of relationships linking entities.

    Look at the JSON schema example response and follow it closely. Pay attention to date and type formats (e.g., EU date format, 24-hour time).
    Ensure that the `source` and `target` nodes in `rels` are the same entities from the `nodes` list, and not different ones. 
    Think about the relationships between the entities, i.e., (node AccidentType occurred_at node Country, or node AccidentType has_date node Date).
    Make sure to map all nodes with other important entities, e.g., (node UniqueAccident has_date node Date, node UniqueAccident occurred_at node Country).
    DO NOT map entities like (node Date is_date to node Time) or (node AccidentType is_type to node Country). This is incorrect.

    Schema example:
    {schema_example}

    Accident report context:
    {text}

    JSON:
    """

In [61]:
def extract_knowledge_graph(text: str) -> Dict[str, Any]:
    """
    Extracts entities & relationships from a railway accident report using GPT.
    - First, counts tokens and allows user decision.
    - If within limit, runs GPT and handles errors.
    """

    # Build prompt
    prompt = build_prompt(text)

    # Call GPT
    response_text = call_gpt(prompt)

    try:
        extracted_graph = json.loads(response_text)  # Ensure valid JSON
        return extracted_graph  # Successfully parsed knowledge graph
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", str(e))
        print("Storing raw response for review...")

        # Save the faulty response for debugging
        with open("failed_graph_extractions.json", "a") as file:
            json.dump({"input_text": text[:1000], "raw_output": response_text}, file, indent=4)
            file.write("\n")

        return {}  # Return empty dictionary in case of failure

In [167]:
# Define token limit for function execution
token_limit = 4096

# Build the prompt and count tokens
prompt = build_prompt(relevant_text)
token_count = count_tokens(prompt)
estimated_cost = token_count * 0.00000015  # Approximate OpenAI pricing

# Check token limit
if token_count > token_limit:
    print(f"Token count is too high: {token_count}\nPlease reduce the chunk size or refine the prompt.")
else:
    print(f"Token count for prompt: {token_count}")

Token count for prompt: 2257


In [ ]:
# Confirm Execution
proceed = input("Do you want to proceed with information extraction? (yes/no): ").strip().lower()
if proceed != "yes":
    print("Extraction aborted by user.")
else:
    print("Sending request to GPT...")
    response_json = extract_knowledge_graph(pdf_text)

response_json

Sending request to GPT...


{'nodes': [{'id': 'Dublin-Cork Incident', 'type': 'UniqueAccident'},
  {'id': 'Broken Rail', 'type': 'AccidentType'},
  {'id': '21/02/2023', 'type': 'Date'},
  {'id': '07:56', 'type': 'Time'},
  {'id': 'Ireland', 'type': 'Country'},
  {'id': 'Commission for Railway Regulation', 'type': 'RegulatoryBody'}],
 'rels': [{'source': 'Dublin-Cork Incident',
   'target': 'Ireland',
   'type': 'occurred_in'},
  {'source': 'Dublin-Cork Incident',
   'target': 'Broken Rail',
   'type': 'is_type'},
  {'source': 'Dublin-Cork Incident',
   'target': 'Commission for Railway Regulation',
   'type': 'investigated_by'},
  {'source': 'Dublin-Cork Incident',
   'target': '21/02/2023',
   'type': 'has_date'},
  {'source': 'Dublin-Cork Incident', 'target': '07:56', 'type': 'has_time'}]}

In [169]:
# Define CSV storage file
CSV_FILE = "pdf_processing_results.csv"

def append_pdf_json_result(pdf_name: str, response_json: dict) -> pd.DataFrame:
    """
    Appends the JSON output of response_json function to a DataFrame.
    
    - If the PDF has been processed before, it appends a **new row** instead of a new column.
    - Prevents duplicate JSON entries for the same iteration.
    - Ensures data is **stored in rows**, making querying and analysis easier.

    Args:
        pdf_name (str): Name of the processed PDF file.
        response_json (dict): JSON response from the knowledge extraction process.

    Returns:
        pd.DataFrame: Updated DataFrame with the new result.
    """
    # Convert JSON response to a formatted string for easy comparison
    json_output = json.dumps(response_json, indent=2)

    # Load existing results if the CSV exists
    if os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE, dtype={"iteration_number": int})
    else:
        # Create an empty DataFrame with the correct schema
        df = pd.DataFrame(columns=["pdf_name", "iteration_number", "json_output"])

    # Filter for the current PDF's past records
    pdf_history = df[df["pdf_name"] == pdf_name]

    # Check for duplicates: If this JSON output already exists for the same PDF, skip re-adding
    if not pdf_history.empty and json_output in pdf_history["json_output"].values:
        print(f"No changes detected in JSON for {pdf_name}, skipping new entry.")
        return df  # Exit early if it's a duplicate

    # Determine new iteration number
    iteration_number = pdf_history["iteration_number"].max() + 1 if not pdf_history.empty else 1

    # Append new result
    new_entry = pd.DataFrame({"pdf_name": [pdf_name], "iteration_number": [iteration_number], "json_output": [json_output]})
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save back to CSV in **append mode** to avoid full file reads/writes
    df.to_csv(CSV_FILE, index=False)

    print(f"Successfully added {pdf_name} - Iteration {iteration_number} to results!")
    return df

# Example execution
results_df = append_pdf_json_result(pdf_name, response_json)

Successfully added IE-10404 - 230222 Broken Rail Emly.pdf - Iteration 1 to results!


In [170]:
results_df

,pdf_name,iteration_number,json_output
0,IE-10397 - 211207 Clontarf.pdf,1,"{\n ""nodes"": [\n {\n ""id"": ""Signal Pa..."
1,IE-10375 - 210827 Collision with track equipme...,1,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
2,IE-10375 - 210827 Collision with track equipme...,2,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
3,IE-10375 - 210827 Collision with track equipme...,3,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
4,IE-10404 - 230222 Broken Rail Emly.pdf,1,"{\n ""nodes"": [\n {\n ""id"": ""Dublin-Co..."


In [171]:
# Defining functions for mapping extracted entities to graph nodes and relationships
def props_to_dict(props) -> dict:
    """Converts properties to a dictionary for graph storage."""
    properties = {}
    if not props:
        return properties
    for p in props:
        properties[p["key"]] = p["value"]
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Maps extracted entities to graph nodes."""
    properties = {"name": node.id}
    return BaseNode(
        id=node.id,
        type=node.type.capitalize(),
        properties=properties
    )

def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Maps extracted relationships to graph edges."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}

    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

# Neo4j Storage

In [96]:
# Neo4j Connection Setup
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

try:
    # Test the connection
    with driver.session() as session:
        session.run("RETURN 1")
    print("Connected to Neo4j successfully.")
except AuthError as e:
    print("Authentication failed. Check your credentials:", str(e))

Connected to Neo4j successfully.


In [147]:
# Clear database
def clear_neo4j_database():
    """Delete all nodes and relationships in the Neo4j database."""
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("Neo4j database cleared successfully.")

# Run the function to clear the database
clear_neo4j_database()

Neo4j database cleared successfully.


In [172]:
results_df

,pdf_name,iteration_number,json_output
0,IE-10397 - 211207 Clontarf.pdf,1,"{\n ""nodes"": [\n {\n ""id"": ""Signal Pa..."
1,IE-10375 - 210827 Collision with track equipme...,1,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
2,IE-10375 - 210827 Collision with track equipme...,2,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
3,IE-10375 - 210827 Collision with track equipme...,3,"{\n ""nodes"": [\n {\n ""id"": ""Train J28..."
4,IE-10404 - 230222 Broken Rail Emly.pdf,1,"{\n ""nodes"": [\n {\n ""id"": ""Dublin-Co..."


In [173]:
def get_json_output(df, pdf_name, iteration_number):
    """
    Gets the 'json_output' for the given pdf_name and iteration_number.
    Returns an empty dict if there's no match.
    """
    subset = df[
        (df["pdf_name"] == pdf_name) &
        (df["iteration_number"] == iteration_number)
    ]

    if subset.empty:
        print("No match found.")
        return {}

    json_str = subset.iloc[0]["json_output"]
    return json.loads(json_str)

# Example usage:
pdf_of_choice= "IE-10404 - 230222 Broken Rail Emly.pdf"
json_to_convert = get_json_output(results_df, pdf_of_choice, 1)
print(json.dumps(json_to_convert, indent=2))

{
  "nodes": [
    {
      "id": "Dublin-Cork Incident",
      "type": "UniqueAccident"
    },
    {
      "id": "Broken Rail",
      "type": "AccidentType"
    },
    {
      "id": "21/02/2023",
      "type": "Date"
    },
    {
      "id": "07:56",
      "type": "Time"
    },
    {
      "id": "Ireland",
      "type": "Country"
    },
    {
      "id": "Commission for Railway Regulation",
      "type": "RegulatoryBody"
    }
  ],
  "rels": [
    {
      "source": "Dublin-Cork Incident",
      "target": "Ireland",
      "type": "occurred_in"
    },
    {
      "source": "Dublin-Cork Incident",
      "target": "Broken Rail",
      "type": "is_type"
    },
    {
      "source": "Dublin-Cork Incident",
      "target": "Commission for Railway Regulation",
      "type": "investigated_by"
    },
    {
      "source": "Dublin-Cork Incident",
      "target": "21/02/2023",
      "type": "has_date"
    },
    {
      "source": "Dublin-Cork Incident",
      "target": "07:56",
      "type": "has_

In [128]:
def convert_json_to_graph(json_to_convert, source_text):
    """
    Converts extracted JSON into a graph-compatible format with correct entity types.
    """

    def get_node_type(json_data, node_id):
        """
        Helper function to retrieve the correct node type from JSON.
        """
        for node in json_data["nodes"]:
            if node["id"] == node_id:
                return node["type"]
        return "Unknown"  # Fallback if type is missing

    if not json_to_convert:
        print("No valid data to convert to a graph.")
        return None

    # Convert Nodes
    graph_nodes = [map_to_base_node(Node(id=node["id"], type=node["type"])) for node in json_to_convert["nodes"]]

    # Convert Relationships (Ensure correct types)
    graph_rels = []
    for rel in json_to_convert["rels"]:
        source_node = Node(id=rel["source"], type=get_node_type(json_to_convert, rel["source"]))
        target_node = Node(id=rel["target"], type=get_node_type(json_to_convert, rel["target"]))
        graph_rels.append(map_to_base_relationship(Relationship(source=source_node, target=target_node, type=rel["type"])))

    return GraphDocument(nodes=graph_nodes, relationships=graph_rels, source=Document(page_content=source_text))

In [129]:
def store_in_neo4j(graph_document):
    """
    Stores extracted knowledge graph into Neo4j with dynamic labels.
    """
    with driver.session() as session:
        # Store nodes with dynamic labels
        for node in graph_document.nodes:
            session.run(f"""
                MERGE (n:{node.type} {{id: $id}})
                ON CREATE SET n.name = $name
            """, id=node.id, name=node.id)

        # Store relationships
        for rel in graph_document.relationships:
            session.run(f"""
                MATCH (s {{id: $source}})
                MATCH (t {{id: $target}})
                MERGE (s)-[r:{rel.type.upper()}]->(t)
            """, source=rel.source.id, target=rel.target.id)

In [101]:
def process_railway_accident_report(json_to_convert):
    """
    Converts the JSON to a graph format and stores it in Neo4j.
    """
    print("Converting JSON to graph format...")
    graph_document = convert_json_to_graph(json_to_convert, relevant_text)

    if graph_document:
        print("Graph structure created! Storing in Neo4j...")
        store_in_neo4j(graph_document)

In [174]:
# Store extracted entities into Neo4j
try:
    db_result = process_railway_accident_report(json_to_convert)
    print("Data stored in Neo4j successfully.")
except Exception as e:
    print("Failed to store data in Neo4j:", str(e))

Converting JSON to graph format...
Graph structure created! Storing in Neo4j...
Data stored in Neo4j successfully.


In [ ]:
# Close Neo4j connection
driver.close()